In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
"""
고객 이탈 여부 예측을 위한 딥러닝 분류 모델 설계
클래스 불균형을 고려한 MLP 기반 이진 분류 모델 구현

고객 데이터를 기반으로 이탈 여부를 예측하는 딥러닝 분류 모델을 설계해야 합니다.
"""

In [26]:
# 데이터 불러오기
df = pd.read_csv('customer_data_balanced.csv')

In [27]:
# 데이터 확인
df.head()

,Age,Tenure,MonthlySpending_KRW,ContractType,CustomerServiceCalls,IsChurn
0,56,29,249010,1,6,0
1,69,44,54542,1,6,0
2,46,53,30651,1,1,0
3,32,24,119239,0,5,1
4,60,58,361075,1,1,0


In [28]:
# ContractType 원-핫 인코딩
df = pd.get_dummies(df, columns=['ContractType'])

In [29]:
# Feature, Target 정의
X = df.drop('IsChurn', axis=1)
y = df['IsChurn']

In [30]:
# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [31]:
# 학습 및 테스트 데이터 설정
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [32]:
# 모델 정의
model = models.Sequential([
    # 뉴런 64개
    # input_shape : 처음 입력 받는 층이면 반드시 입력 데이터의 형태(shape)를 명시해야 함
    # X_train.shape[1] : 열의 수
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),   # 입력 데이터의 차원 : 1차원 튜플
    layers.Dropout(0.3),   
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid') # 이진 분류
])

model.compile(optimizer='adam',             # 최적화 알고리즘
              loss='binary_crossentropy',   # 손실함수 : 이진 분류에서 많이 씀
              metrics=['accuracy'])         # 모델 성능 평가 기준

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# 클래스 가중치
# 이탈 고객을 2배 더 중요하게 학습
class_weights = {0: 1.0, 1: 2.0}

# EarlyStopping 콜백 정의
# monitor : 검증 손실이
# patience : 5번 연속 개선되지 않으면
# restore_best_weights : 가장 성능 좋았던 가중치로 복원
# 과적합 방지하고 학습시간 절약
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
history = model.fit(X_train, y_train,
                    epochs=50,                      
                    batch_size=32,                  # 데이터 32개씩 나눠서 한 번에 학습
                    validation_split=0.2,           # 훈련 데이터의 20%를 검증용으로 자동 분리
                    class_weight=class_weights,     # 클래스 가중치
                    callbacks=[early_stop],         # 학습 성능 보고 자동 종료
                    verbose=1)                      # 모델이 학습하면서 진행 상황을 얼마나 상세하게 보여줄지 조절하는 옵션

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5445 - loss: 0.9745 - val_accuracy: 0.4219 - val_loss: 0.7001
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5047 - loss: 0.9366 - val_accuracy: 0.5750 - val_loss: 0.6758
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5352 - loss: 0.9022 - val_accuracy: 0.5750 - val_loss: 0.6700
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5540 - loss: 0.8898 - val_accuracy: 0.6250 - val_loss: 0.6543
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5687 - loss: 0.8928 - val_accuracy: 0.6156 - val_loss: 0.6536
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5740 - loss: 0.8875 - val_accuracy: 0.6062 - val_loss: 0.6601
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5830 - loss: 0.8799 - val_accuracy: 0.6125 - val_loss: 0.6508
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5679 - loss: 0.8948 - val_accuracy: 0.6031 - val_loss:

In [36]:
# 예측
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# 평가 지표 출력
print('Confusion Matrix : \n', confusion_matrix(y_test, y_pred))
print('\nClassification Report : \n', classification_report(y_test, y_pred))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Confusion Matrix : 
 [[157  99]
 [ 59  85]]

Classification Report : 
               precision    recall  f1-score   support

           0       0.73      0.61      0.67       256
           1       0.46      0.59      0.52       144

    accuracy                           0.60       400
   macro avg       0.59      0.60      0.59       400
weighted avg       0.63      0.60      0.61       400



### 모델 성능 요약 보고서
1. Confusion Matrix
- 실제 이탈 고객이 아님에도 불구하고 이탈로 잘못 분류한 사례 99건 있음 ➡️ 불필요한 마케팅 비용 가능성 있음
- 실제 이탈 고객 59명 놓침

2. Classification Report
- 이탈 고객 예측의 정밀도가 낮은 편 ➡️ 오탐 높은 편
- 실제 이탈 고객의 59% 탐지 ➡️ 이탈 감지 성능 보통
- 이탈 고객 F1 점수 낮은 편

**이탈 고객을 일정 수준 감지할 수 있는 재현율(Recall)을 확보했으나 정밀도(Precision)가 낮아 개선 필요함**